# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f4af86abd00>
├── 'a' --> tensor([[ 1.4714,  0.4159,  0.2250],
│                   [ 1.5735,  0.4514, -1.3711]])
└── 'x' --> <FastTreeValue 0x7f4af86ab460>
    └── 'c' --> tensor([[-0.8511, -0.3600, -2.3103,  0.4098],
                        [ 0.0646,  1.3565, -0.1455,  0.1558],
                        [ 1.2187,  0.0805, -0.0962,  0.9615]])

In [4]:
t.a

tensor([[ 1.4714,  0.4159,  0.2250],
        [ 1.5735,  0.4514, -1.3711]])

In [5]:
%timeit t.a

64.7 ns ± 0.0622 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f4af86abd00>
├── 'a' --> tensor([[ 1.3783, -1.8372,  0.8782],
│                   [ 0.7392, -1.6578,  0.7439]])
└── 'x' --> <FastTreeValue 0x7f4af86ab460>
    └── 'c' --> tensor([[-0.8511, -0.3600, -2.3103,  0.4098],
                        [ 0.0646,  1.3565, -0.1455,  0.1558],
                        [ 1.2187,  0.0805, -0.0962,  0.9615]])

In [7]:
%timeit t.a = new_value

61.2 ns ± 0.0735 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.4714,  0.4159,  0.2250],
               [ 1.5735,  0.4514, -1.3711]]),
    x: Batch(
           c: tensor([[-0.8511, -0.3600, -2.3103,  0.4098],
                      [ 0.0646,  1.3565, -0.1455,  0.1558],
                      [ 1.2187,  0.0805, -0.0962,  0.9615]]),
       ),
)

In [10]:
b.a

tensor([[ 1.4714,  0.4159,  0.2250],
        [ 1.5735,  0.4514, -1.3711]])

In [11]:
%timeit b.a

55.9 ns ± 0.0354 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.3949,  0.6447,  0.9471],
               [-1.1940, -0.1257, -0.3139]]),
    x: Batch(
           c: tensor([[-0.8511, -0.3600, -2.3103,  0.4098],
                      [ 0.0646,  1.3565, -0.1455,  0.1558],
                      [ 1.2187,  0.0805, -0.0962,  0.9615]]),
       ),
)

In [13]:
%timeit b.a = new_value

484 ns ± 0.153 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

846 ns ± 0.168 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 13.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

142 µs ± 298 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

147 µs ± 16.6 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f4a4c36ae20>
├── 'a' --> tensor([[[ 1.4714,  0.4159,  0.2250],
│                    [ 1.5735,  0.4514, -1.3711]],
│           
│                   [[ 1.4714,  0.4159,  0.2250],
│                    [ 1.5735,  0.4514, -1.3711]],
│           
│                   [[ 1.4714,  0.4159,  0.2250],
│                    [ 1.5735,  0.4514, -1.3711]],
│           
│                   [[ 1.4714,  0.4159,  0.2250],
│                    [ 1.5735,  0.4514, -1.3711]],
│           
│                   [[ 1.4714,  0.4159,  0.2250],
│                    [ 1.5735,  0.4514, -1.3711]],
│           
│                   [[ 1.4714,  0.4159,  0.2250],
│                    [ 1.5735,  0.4514, -1.3711]],
│           
│                   [[ 1.4714,  0.4159,  0.2250],
│                    [ 1.5735,  0.4514, -1.3711]],
│           
│                   [[ 1.4714,  0.4159,  0.2250],
│                    [ 1.5735,  0.4514, -1.3711]]])
└── 'x' --> <FastTreeValue 0x7f4a4c36adc0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.9 µs ± 63.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f4a4c36a730>
├── 'a' --> tensor([[ 1.4714,  0.4159,  0.2250],
│                   [ 1.5735,  0.4514, -1.3711],
│                   [ 1.4714,  0.4159,  0.2250],
│                   [ 1.5735,  0.4514, -1.3711],
│                   [ 1.4714,  0.4159,  0.2250],
│                   [ 1.5735,  0.4514, -1.3711],
│                   [ 1.4714,  0.4159,  0.2250],
│                   [ 1.5735,  0.4514, -1.3711],
│                   [ 1.4714,  0.4159,  0.2250],
│                   [ 1.5735,  0.4514, -1.3711],
│                   [ 1.4714,  0.4159,  0.2250],
│                   [ 1.5735,  0.4514, -1.3711],
│                   [ 1.4714,  0.4159,  0.2250],
│                   [ 1.5735,  0.4514, -1.3711],
│                   [ 1.4714,  0.4159,  0.2250],
│                   [ 1.5735,  0.4514, -1.3711]])
└── 'x' --> <FastTreeValue 0x7f4a52f7e850>
    └── 'c' --> tensor([[-0.8511, -0.3600, -2.3103,  0.4098],
                        [ 0.0646,  1.3565, -0.1455,  0.1558],
                 

In [23]:
%timeit t_cat(trees)

30.3 µs ± 49.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.5 µs ± 94.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.8511, -0.3600, -2.3103,  0.4098],
                       [ 0.0646,  1.3565, -0.1455,  0.1558],
                       [ 1.2187,  0.0805, -0.0962,  0.9615]],
              
                      [[-0.8511, -0.3600, -2.3103,  0.4098],
                       [ 0.0646,  1.3565, -0.1455,  0.1558],
                       [ 1.2187,  0.0805, -0.0962,  0.9615]],
              
                      [[-0.8511, -0.3600, -2.3103,  0.4098],
                       [ 0.0646,  1.3565, -0.1455,  0.1558],
                       [ 1.2187,  0.0805, -0.0962,  0.9615]],
              
                      [[-0.8511, -0.3600, -2.3103,  0.4098],
                       [ 0.0646,  1.3565, -0.1455,  0.1558],
                       [ 1.2187,  0.0805, -0.0962,  0.9615]],
              
                      [[-0.8511, -0.3600, -2.3103,  0.4098],
                       [ 0.0646,  1.3565, -0.1455,  0.1558],
                       [ 1.2187,  0.0805, -0.0962,  0.9615]],

In [26]:
%timeit Batch.stack(batches)

131 µs ± 263 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.8511, -0.3600, -2.3103,  0.4098],
                      [ 0.0646,  1.3565, -0.1455,  0.1558],
                      [ 1.2187,  0.0805, -0.0962,  0.9615],
                      [-0.8511, -0.3600, -2.3103,  0.4098],
                      [ 0.0646,  1.3565, -0.1455,  0.1558],
                      [ 1.2187,  0.0805, -0.0962,  0.9615],
                      [-0.8511, -0.3600, -2.3103,  0.4098],
                      [ 0.0646,  1.3565, -0.1455,  0.1558],
                      [ 1.2187,  0.0805, -0.0962,  0.9615],
                      [-0.8511, -0.3600, -2.3103,  0.4098],
                      [ 0.0646,  1.3565, -0.1455,  0.1558],
                      [ 1.2187,  0.0805, -0.0962,  0.9615],
                      [-0.8511, -0.3600, -2.3103,  0.4098],
                      [ 0.0646,  1.3565, -0.1455,  0.1558],
                      [ 1.2187,  0.0805, -0.0962,  0.9615],
                      [-0.8511, -0.3600, -2.3103,  0.4098],
                   

In [28]:
%timeit Batch.cat(batches)

145 µs ± 244 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

309 µs ± 731 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
